In [2]:
import json
import os
from supabase import create_client, Client

file = open("./config.json", "r")
config = json.load(file)

URL = config["SUPABASE_URL"]
KEY = config["SUPABASE_KEY"]

url: str = URL
key: str = KEY
supabase: Client = create_client(url, key)

In [3]:
response = (
    supabase.table("courses")
    .select("id", "code")
    .execute()
)

In [4]:
dict_ = {}
for course in response.data:
    dict_[course['code']] = course['id']

dict_

{'CS-401': '8301f629-ab1e-47d8-ada9-89bac5a138d4',
 'CS-433': '3db123b7-b5e1-45e0-8063-7953fcfb815b',
 'CS-450': 'e3d0b867-b78f-460b-bf29-7250a6d43654',
 'CS-452': 'a7ab7cba-d4be-42ef-b645-13b732c29e4b',
 'CS-451': 'a2af3de1-fec0-4d6b-98aa-c89c096bf786',
 'COM-402': '16b50388-ab38-4bdf-990b-7c93c8e0440f',
 'COM-407': '703a3a4d-7e45-4977-951c-158d376b4c56',
 'COM-401': 'e55e10bb-3c46-4346-99d5-7d978de2f3f8',
 'CS-440': 'a7560307-3cb7-426a-8a7e-13fb56cfc0aa',
 'CS-438': '665da4a7-ebe2-48b7-8ecb-9dfce29e4bbb',
 'CS-442': '78b76966-9a36-475e-9144-192be8e40147',
 'CS-460': '21628cfb-3d6b-4dac-bc0c-8ad1901be69d'}

In [21]:
import pandas as pd
df = pd.read_csv('data/courses_comments.csv')
data = df.groupby('course_code')['comment'].count().sort_values(ascending=False).head(10)
most_rated_courses_comments = df[df['course_code'].isin(data.index)]

In [22]:
most_rated_courses_comments

,course_code,course_link,course_name,overall,prof_rating,difficulty,workload,professors,comment
197,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],"intéressant, bon prof, beaucoup de boulot, fin..."
198,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],"ils ont un grading system assez spécial, genre..."
199,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],If you want a challenging but interesting cour...
200,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],"If you like cryptography, it’s a must do. Howe..."
201,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],If you are interested by the math behind crypt...
...,...,...,...,...,...,...,...,...,...
1149,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']",yes if you're lacking the basic concepts of im...
1150,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']",Basics of image processing. 3 credits pretty e...
1151,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']","Easy class, especially if you already attended..."
1152,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']",Easy course but exam a bit long. It's only the...


In [23]:
for index, row in most_rated_courses_comments.iterrows():
    course_code = row['course_code']
    id = dict_.get(course_code)
    if id is not None:
        response = (
            supabase.table("posts")
            .insert({"course_id": id, "content": row["comment"], "is_anonymous": True})
            .execute()
        )